In [1]:
import torch

In [2]:
# Dataset Preprocessing
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import time
from torchvision import datasets, transforms
import requests
from PIL import Image

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import math
import random
from sklearn.metrics import confusion_matrix

In [3]:
torch.cuda.empty_cache()

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
best_acc = 0
start_epoch = 0

In [6]:
print("==> Preparing data..")
transform_train = transforms.Compose([
    transforms.RandomCrop(32,padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914,0.4822,0.4465),(0.2023,0.1994,0.2010)),
])

==> Preparing data..


In [7]:
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914,0.4822,0.4465),(0.2023,0.1994,0.2010)),
])

In [8]:
trainset = torchvision.datasets.CIFAR10(
root='./data',train=True,download=True,transform=transform_train)

Files already downloaded and verified


In [9]:
torchloader = torch.utils.data.DataLoader(
trainset,batch_size=128,shuffle=True,num_workers=2)

In [10]:
testset = torchvision.datasets.CIFAR10(
root='./data',train=False,download=True,transform=transform_test)

Files already downloaded and verified


In [11]:
testloader = torch.utils.data.DataLoader(testset,batch_size=100,shuffle=False,num_workers=2)

In [12]:
classes = ('plane','car','bird','cat','deer','dog','frog','horse','ship','truck')

In [15]:
from cifar10_models.resnet import resnet18
my_model = resnet18()

# Pretrained model
my_model = resnet18(pretrained=True)
my_model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [16]:
epoch= 1

In [17]:
criterion = nn.CrossEntropyLoss()

In [18]:
def test(epochs):
    my_model.eval()
    val_acc=0
    val_loss=0
    val_correct=0
    val_total = 0
    with torch.no_grad():
        for batch_idx, (inputs,targets) in enumerate(testloader):
            inputs,targets = inputs.to(device),targets.to(device)
            outputs = my_model(inputs)
            loss = criterion(outputs,targets)
            val_loss += loss.item()
            _,predicted = outputs.max(1)
            val_total += targets.size(0)
            val_correct += predicted.eq(targets).sum().item()
            val_acc = 100.*val_correct/val_total
            if batch_idx%40==39:
                print("Evaluating....")
                print(batch_idx,len(testloader),'Loss: %.3f | Acc: %0.3f (%d/%d)' % (val_loss/(batch_idx+1),100*val_correct/val_total,val_correct,val_total))
    

In [19]:
for e in range(epoch):
    print("Epoch", e+1)
    test(epoch)

Epoch 1
Evaluating....
39 100 Loss: 0.283 | Acc: 92.125 (3685/4000)
Evaluating....
79 100 Loss: 0.276 | Acc: 92.475 (7398/8000)
